# 06_portfolio_advies - Pon Brand Portfolio Strategy

Deze notebook analyseert Pon's fietsmerken portfolio en beantwoordt de kernvraag:
**"Should Pon stop/add brands (with CSR/political considerations)?"**

**Analyse scope:**
1. Brand performance comparison (dealers, ratings, coverage)
2. Market positioning en overlap analysis
3. Cannibalisatie tussen Pon merken 
4. CSR/sustainability impact per brand
5. Stop/add/pivot aanbevelingen met rationale

**Input**: Dealer data, brand performance metrics, policy trends
**Output**: Data-driven portfolio strategy recommendations voor board

In [49]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Directories
DATA_DIR = Path('../data')
OUTPUTS_DIR = Path('../outputs')
OUTPUTS_DIR.mkdir(exist_ok=True)
(OUTPUTS_DIR / 'tables').mkdir(exist_ok=True)
(OUTPUTS_DIR / 'plots').mkdir(exist_ok=True)

# Pon brand portfolio - CORRECTED to match actual data format
PON_BRANDS = {
    'gazelle', 'cannondale', 'union', 'kalkhoff', 
    'urban_arrow',  # underscore format (correct)
    'cervélo', 'cervelo', 'focus', 'santa_cruz'
}

print("✅ Setup complete - Pon Brand Portfolio Strategy Analysis")
print(f"Working directory: {Path.cwd()}")
print(f"🔧 CORRECTED Pon brands set: {sorted(PON_BRANDS)}")

✅ Setup complete - Pon Brand Portfolio Strategy Analysis
Working directory: /Users/DINGZEEFS/Case_Gazelle_Pon/notebooks
🔧 CORRECTED Pon brands set: ['cannondale', 'cervelo', 'cervélo', 'focus', 'gazelle', 'kalkhoff', 'santa_cruz', 'union', 'urban_arrow']


## Load Data & Brand Performance Analysis

In [50]:
# Load processed dealer data - USE CORRECTED DATA
dealers = pd.read_parquet(DATA_DIR / 'processed/dealers.parquet')  # Location-based (for coverage)
dealers_all_brands = pd.read_parquet(DATA_DIR / 'processed/dealers_all_brands.parquet')  # All brand relationships

print(f"📊 Dataset Overview (CORRECTED):")
print(f"   Unique dealer locations: {len(dealers):,}")
print(f"   Total brand relationships: {len(dealers_all_brands):,}")
print(f"   Multi-brand locations: {(dealers['total_brand_count'] > 1).sum():,}")
print(f"   Pon brand relationships: {dealers_all_brands['is_pon_dealer'].sum():,}")
print(f"   Pon locations: {dealers['is_pon_dealer'].sum():,}")

# TRUE market share calculation
true_market_share = dealers_all_brands['is_pon_dealer'].sum() / len(dealers_all_brands) * 100
location_market_share = dealers['is_pon_dealer'].sum() / len(dealers) * 100

print(f"\\n📈 TRUE Market Share:")
print(f"   By relationships: {true_market_share:.1f}% (was artificially 43.8%)")
print(f"   By locations: {location_market_share:.1f}%")

# CORRECTED Brand performance metrics using all brand relationships
brand_performance = dealers_all_brands[dealers_all_brands['is_pon_dealer']].groupby('brand_clean').agg({
    'google_place_id': 'nunique',  # Unique locations per brand  
    'name': 'count',  # Total brand relationships
    'google_rating': 'mean',
    'google_user_ratings_total': 'mean'
}).reset_index()

brand_performance.columns = ['brand_clean', 'unique_locations', 'total_relationships', 'avg_rating', 'avg_reviews']

# Calculate TRUE performance score
brand_performance['performance_score'] = (
    brand_performance['avg_rating'] * 
    np.log1p(brand_performance['avg_reviews']) * 
    np.log1p(brand_performance['total_relationships'])  # Use total relationships
)

# Market presence metrics
brand_performance['location_presence_pct'] = (
    brand_performance['unique_locations'] / len(dealers) * 100
)
brand_performance['relationship_share_pct'] = (
    brand_performance['total_relationships'] / dealers_all_brands['is_pon_dealer'].sum() * 100
)

brand_performance = brand_performance.sort_values('total_relationships', ascending=False)

print(f"\\n🎯 CORRECTED Pon Brand Performance:")
display_cols = ['brand_clean', 'unique_locations', 'total_relationships', 'location_presence_pct', 'avg_rating']
print(brand_performance[display_cols].round(1).to_string(index=False))

print(f"\\n💡 Key Corrections:")
print(f"   Union: Now shows true performance (was severely underestimated)")
print(f"   Market share: Realistic {true_market_share:.1f}% vs inflated 43.8%") 
print(f"   Cannibalization: Now detectable with {(dealers['total_brand_count'] > 1).sum():,} multi-brand locations")

📊 Dataset Overview (CORRECTED):
   Unique dealer locations: 2,080
   Total brand relationships: 6,748
   Multi-brand locations: 1,374
   Pon brand relationships: 1,507
   Pon locations: 978
\n📈 TRUE Market Share:
   By relationships: 22.3% (was artificially 43.8%)
   By locations: 47.0%
\n🎯 CORRECTED Pon Brand Performance:
brand_clean  unique_locations  total_relationships  location_presence_pct  avg_rating
    gazelle               700                  701                   33.7         4.5
      union               247                  247                   11.9         4.4
urban_arrow               211                  213                   10.1         4.4
   kalkhoff               191                  199                    9.2         4.5
 cannondale                92                   92                    4.4         4.6
    cervelo                35                   35                    1.7         4.6
 santa_cruz                12                   12                    0.6

## Market Positioning & Overlap Analysis

In [51]:
# Analyze market positioning by dealer density and demographics - CORRECTED
# Get PC4-level analysis using BRAND RELATIONSHIPS not just locations
print("🔧 CORRECTED: Using brand relationships for dealer counts")

# FIXED: Merge PC4 column from dealers dataset FIRST
dealers_with_pc4 = dealers_all_brands.merge(
    dealers[['google_place_id', 'pc4']], 
    on='google_place_id', 
    how='left'
)

print(f"After PC4 merge: {dealers_with_pc4.shape}")
print(f"PC4 column available: {'pc4' in dealers_with_pc4.columns}")

# Now group by PC4 and brand using the merged dataset with PC4 column
dealer_pc4_relationships = dealers_with_pc4.groupby(['pc4', 'brand_clean']).size().reset_index(name='relationships_per_pc4')

# Also get location counts for comparison using the same merged dataset
dealer_pc4_locations = dealers_with_pc4.groupby(['pc4', 'brand_clean'])['google_place_id'].nunique().reset_index(name='locations_per_pc4')

print(f"   Relationships data shape: {dealer_pc4_relationships.shape}")
print(f"   Locations data shape: {dealer_pc4_locations.shape}")

# Load demografie data
demografie = pd.read_parquet(DATA_DIR / 'processed/demografie.parquet')
print("Available columns in demografie:", list(demografie.columns))

# Use available columns only
demographic_summary = demografie.groupby('pc4').agg({
    'pop_total': 'first',
    'density': 'first', 
    'woz': 'first',
    'kids_0_15_pct': 'first',
    'age_25_44_pct': 'first'
}).reset_index()

# Estimate singles percentage (crude approximation)
demographic_summary['hh_1p_pct'] = 0.35 - demographic_summary['kids_0_15_pct']

# Merge for brand-demographic analysis - USE RELATIONSHIPS DATA
brand_demo = dealer_pc4_relationships.merge(demographic_summary, on='pc4', how='left')
brand_demo = brand_demo[brand_demo['brand_clean'].isin(PON_BRANDS)]

print(f"\n📊 CORRECTED Brand-demographic data shape: {brand_demo.shape}")
print(f"Brands found: {sorted(brand_demo['brand_clean'].unique())}")

# Calculate weighted averages per brand (weighted by RELATIONSHIP count)
brand_positioning = []
for brand in PON_BRANDS:
    brand_data = brand_demo[brand_demo['brand_clean'] == brand]
    if len(brand_data) > 0:
        weights = brand_data['relationships_per_pc4']  # Use relationships as weights
        positioning = {
            'brand': brand,
            'total_relationships': weights.sum(),  # CORRECTED: total relationships
            'total_locations': dealer_pc4_locations[dealer_pc4_locations['brand_clean'] == brand]['locations_per_pc4'].sum() if brand in dealer_pc4_locations['brand_clean'].values else 0,
            'avg_density': np.average(brand_data['density'].fillna(0), weights=weights),
            'avg_woz': np.average(brand_data['woz'].fillna(0), weights=weights),
            'avg_kids_pct': np.average(brand_data['kids_0_15_pct'].fillna(0), weights=weights),
            'avg_families_pct': np.average(brand_data['age_25_44_pct'].fillna(0), weights=weights),
            'avg_singles_pct': np.average(brand_data['hh_1p_pct'].fillna(0.3), weights=weights)
        }
        brand_positioning.append(positioning)

positioning_df = pd.DataFrame(brand_positioning)
if not positioning_df.empty:
    positioning_df = positioning_df.sort_values('total_relationships', ascending=False)
    
    print("🎯 CORRECTED Brand Market Positioning (weighted by relationship presence):")
    print(positioning_df.round(3).to_string(index=False))
    
    # Market segments based on demographics
    positioning_df['market_segment'] = 'Mid-market'
    positioning_df.loc[positioning_df['avg_woz'] > 400, 'market_segment'] = 'Premium'
    positioning_df.loc[positioning_df['avg_woz'] < 250, 'market_segment'] = 'Budget'
    positioning_df.loc[positioning_df['avg_kids_pct'] > 0.18, 'market_segment'] = 'Family'
    positioning_df.loc[positioning_df['avg_density'] < 1000, 'market_segment'] = 'Rural'
    positioning_df.loc[positioning_df['avg_density'] > 3000, 'market_segment'] = 'Urban'
    
    print(f"\n📊 CORRECTED Market Segmentation:")
    print(positioning_df[['brand', 'market_segment', 'total_relationships', 'total_locations']].to_string(index=False))
    
    print(f"\n💡 Key Corrections:")
    print(f"   Urban Arrow: {positioning_df[positioning_df['brand'] == 'urban_arrow']['total_relationships'].iloc[0] if 'urban_arrow' in positioning_df['brand'].values else 'Not found'} relationships vs {positioning_df[positioning_df['brand'] == 'urban_arrow']['total_locations'].iloc[0] if 'urban_arrow' in positioning_df['brand'].values else 'Not found'} locations")
    print(f"   Gazelle: {positioning_df[positioning_df['brand'] == 'gazelle']['total_relationships'].iloc[0] if 'gazelle' in positioning_df['brand'].values else 'Not found'} relationships vs {positioning_df[positioning_df['brand'] == 'gazelle']['total_locations'].iloc[0] if 'gazelle' in positioning_df['brand'].values else 'Not found'} locations")
    
else:
    print("⚠️ No positioning data found - check brand name mapping")
    positioning_df = pd.DataFrame(columns=['brand', 'total_relationships', 'total_locations', 'avg_density', 'avg_woz', 'avg_kids_pct', 'avg_families_pct', 'avg_singles_pct', 'market_segment'])

🔧 CORRECTED: Using brand relationships for dealer counts
After PC4 merge: (6748, 18)
PC4 column available: True
   Relationships data shape: (6014, 3)
   Locations data shape: (6014, 3)
Available columns in demografie: ['pc4', 'gemeente', 'pop_total', 'hh_total', 'kids_0_15', 'age_25_44', 'koop_pct', 'huur_pct', 'density', 'woz', 'kids_0_15_pct', 'age_25_44_pct', 'income_norm', 'density_norm', 'cluster']

📊 CORRECTED Brand-demographic data shape: (1424, 9)
Brands found: ['cannondale', 'cervelo', 'focus', 'gazelle', 'kalkhoff', 'santa_cruz', 'union', 'urban_arrow']
🎯 CORRECTED Brand Market Positioning (weighted by relationship presence):
      brand  total_relationships  total_locations  avg_density  avg_woz  avg_kids_pct  avg_families_pct  avg_singles_pct
    gazelle                  701              700     1539.398  367.949         0.148             0.237            0.202
      union                  247              247     2476.794  394.417         0.144             0.266          

## Cannibalization Analysis

In [52]:
# CORRECTED: Cannibalization analysis using fixed multi-brand data
print("🔍 CORRECTED Cannibalization Analysis:")
print("=" * 50)

# Use location data with preserved multi-brand information
multi_brand_locations = dealers[dealers['total_brand_count'] > 1].copy()
multi_pon_locations = dealers[dealers['pon_brand_count'] > 1].copy()

print(f"📊 Multi-brand Statistics:")
print(f"   Total dealer locations: {len(dealers):,}")
print(f"   Multi-brand locations: {len(multi_brand_locations):,} ({len(multi_brand_locations)/len(dealers)*100:.1f}%)")
print(f"   Multi-Pon-brand locations: {len(multi_pon_locations):,} ({len(multi_pon_locations)/len(dealers)*100:.1f}%)")
print(f"   Average brands per multi-brand location: {multi_brand_locations['total_brand_count'].mean():.1f}")

# Cannibalization rate for Pon dealers specifically
pon_locations = dealers[dealers['is_pon_dealer']]
pon_cannibalization_rate = len(multi_pon_locations) / len(pon_locations) * 100

print(f"\\n🎯 Pon Brand Cannibalization:")
print(f"   Pon dealer locations: {len(pon_locations):,}")
print(f"   Locations with multiple Pon brands: {len(multi_pon_locations):,}")
print(f"   Pon cannibalization rate: {pon_cannibalization_rate:.1f}%")

# Most common Pon brand combinations
print(f"\\n📈 Most Common Multi-Pon Brand Combinations:")
if len(multi_pon_locations) > 0:
    # Extract only Pon brands from brands_sold
    multi_pon_locations['pon_brands_only'] = multi_pon_locations['brands_sold'].apply(
        lambda x: ', '.join([b for b in x.split(', ') if b in PON_BRANDS])
    )
    
    pon_combos = multi_pon_locations['pon_brands_only'].value_counts().head(10)
    for combo, count in pon_combos.items():
        print(f"   {combo}: {count} locations")
        
    # Show sample multi-Pon dealers
    print(f"\\n🏪 Sample Multi-Pon Dealers:")
    sample_cols = ['name', 'pon_brands_only', 'pon_brand_count', 'postal_code']
    sample = multi_pon_locations[sample_cols].head(5)
    print(sample.to_string(index=False))
else:
    print("   No multi-Pon locations found")

# Brand co-occurrence analysis (Pon brands only)
print(f"\\n🎯 Pon Brand Co-occurrence Analysis:")
brand_pairs = []
for _, row in multi_pon_locations.iterrows():
    pon_brands_at_location = [b for b in row['brands_sold'].split(', ') if b in PON_BRANDS]
    for i, brand1 in enumerate(pon_brands_at_location):
        for brand2 in pon_brands_at_location[i+1:]:
            brand_pairs.append({'brand1': brand1, 'brand2': brand2})

if brand_pairs:
    cooccurrence = pd.DataFrame(brand_pairs)
    cooccurrence_counts = cooccurrence.groupby(['brand1', 'brand2']).size().reset_index(name='cooccurrence_count')
    cooccurrence_counts = cooccurrence_counts.sort_values('cooccurrence_count', ascending=False)
    
    print(cooccurrence_counts.head(10).to_string(index=False))
    
    # Cannibalization insights
    total_cooccurrences = cooccurrence_counts['cooccurrence_count'].sum()
    top_pair = cooccurrence_counts.iloc[0]
    
    print(f"\\n📊 Cannibalization Insights:")
    print(f"   Total Pon brand co-locations: {total_cooccurrences}")
    print(f"   Highest risk pair: {top_pair['brand1']} + {top_pair['brand2']} ({top_pair['cooccurrence_count']} locations)")
    print(f"   Cannibalization impact: {pon_cannibalization_rate:.1f}% of Pon locations have internal competition")
    
else:
    print("No Pon brand co-occurrences detected")
    
print(f"\\n✅ Cannibalization analysis now working with real multi-brand data!")

🔍 CORRECTED Cannibalization Analysis:
📊 Multi-brand Statistics:
   Total dealer locations: 2,080
   Multi-brand locations: 1,374 (66.1%)
   Multi-Pon-brand locations: 391 (18.8%)
   Average brands per multi-brand location: 4.2
\n🎯 Pon Brand Cannibalization:
   Pon dealer locations: 978
   Locations with multiple Pon brands: 391
   Pon cannibalization rate: 40.0%
\n📈 Most Common Multi-Pon Brand Combinations:
   gazelle, union: 111 locations
   gazelle, urban_arrow: 58 locations
   gazelle, kalkhoff: 52 locations
   gazelle, kalkhoff, union: 32 locations
   gazelle, union, urban_arrow: 29 locations
   kalkhoff, urban_arrow: 15 locations
   gazelle, kalkhoff, urban_arrow: 12 locations
   cannondale, gazelle: 11 locations
   cannondale, gazelle, urban_arrow: 7 locations
   cannondale, gazelle, union: 7 locations
\n🏪 Sample Multi-Pon Dealers:
                                      name                  pon_brands_only  pon_brand_count postal_code
                    Fietswinkel Gelderland   

## CSR & Sustainability Impact Analysis

In [53]:
# CSR/Sustainability impact per brand based on:
# 1. Zero-emission zone presence (policy alignment)
# 2. Urban density (sustainable transport impact)
# 3. Family market presence (societal benefit)

# Load ZE-zone and policy data
try:
    ze_steden = pd.read_csv(DATA_DIR / 'external/ze_steden.csv')
    policy_index = pd.read_csv(OUTPUTS_DIR / 'tables/policy_index.csv')
    print(f"📋 Policy data loaded: {len(ze_steden)} ZE cities, {len(policy_index)} policy scores")
except:
    print("⚠️ Policy data not found - using demographic proxies only")
    ze_steden = pd.DataFrame()
    policy_index = pd.DataFrame()

# Calculate CSR scores per brand
csr_analysis = positioning_df.copy()

# CSR Score Components (0-10 scale each)
# 1. Urban impact (higher density = more sustainable transport impact)
max_density = csr_analysis['avg_density'].max()
csr_analysis['urban_impact_score'] = (csr_analysis['avg_density'] / max_density) * 10

# 2. Family benefit (higher kids % = more family-friendly)
max_kids = csr_analysis['avg_kids_pct'].max()
csr_analysis['family_benefit_score'] = (csr_analysis['avg_kids_pct'] / max_kids) * 10

# 3. Accessibility (lower WOZ = more accessible to diverse income levels)
max_woz = csr_analysis['avg_woz'].max()
min_woz = csr_analysis['avg_woz'].min()
csr_analysis['accessibility_score'] = (1 - (csr_analysis['avg_woz'] - min_woz) / (max_woz - min_woz)) * 10

# 4. Policy alignment (based on brand positioning)
csr_analysis['policy_alignment_score'] = 5.0  # baseline
# Urban Arrow gets highest policy score (cargo bikes in ZE zones)
csr_analysis.loc[csr_analysis['brand'] == 'urban_arrow', 'policy_alignment_score'] = 10.0
# E-bike brands get higher scores
ebike_brands = ['gazelle', 'kalkhoff']  # typically e-bike focused
csr_analysis.loc[csr_analysis['brand'].isin(ebike_brands), 'policy_alignment_score'] = 8.0
# Sport brands get lower policy alignment
sport_brands = ['cannondale', 'cervelo', 'santa_cruz', 'focus']
csr_analysis.loc[csr_analysis['brand'].isin(sport_brands), 'policy_alignment_score'] = 3.0

# Calculate overall CSR score
weights = {
    'urban_impact': 0.3,
    'family_benefit': 0.2, 
    'accessibility': 0.2,
    'policy_alignment': 0.3
}

csr_analysis['csr_score'] = (
    weights['urban_impact'] * csr_analysis['urban_impact_score'] +
    weights['family_benefit'] * csr_analysis['family_benefit_score'] +
    weights['accessibility'] * csr_analysis['accessibility_score'] +
    weights['policy_alignment'] * csr_analysis['policy_alignment_score']
)

csr_analysis = csr_analysis.sort_values('csr_score', ascending=False)

print(f"🌱 Brand CSR/Sustainability Scores:")
csr_display = csr_analysis[['brand', 'csr_score', 'urban_impact_score', 
                           'family_benefit_score', 'accessibility_score', 
                           'policy_alignment_score']].round(1)
print(csr_display.to_string(index=False))

📋 Policy data loaded: 29 ZE cities, 29 policy scores
🌱 Brand CSR/Sustainability Scores:
      brand  csr_score  urban_impact_score  family_benefit_score  accessibility_score  policy_alignment_score
   kalkhoff        7.7                 5.1                   9.2                  9.5                     8.0
    gazelle        7.6                 4.5                   9.8                  9.5                     8.0
      focus        7.3                10.0                   7.0                 10.0                     3.0
urban_arrow        7.1                 7.6                   9.3                  0.0                    10.0
      union        6.7                 7.2                   9.6                  5.6                     5.0
 cannondale        5.5                 5.4                   9.3                  5.7                     3.0
    cervelo        5.2                 5.0                  10.0                  4.2                     3.0
 santa_cruz        5.2          

## Portfolio Strategy Recommendations

In [54]:
# Combine all analysis for final recommendations - CORRECTED to use relationship counts
# Use brand_performance data as base (it has the corrected metrics)
final_analysis = brand_performance.copy()

print("🔧 CORRECTED: Using relationship counts for strategic analysis")
print(f"   Brand performance data shape: {brand_performance.shape}")
print(f"   Brands in analysis: {sorted(brand_performance['brand_clean'].unique())}")

# Use total_relationships as dealer_count (CORRECTED)
final_analysis.rename(columns={'total_relationships': 'dealer_count'}, inplace=True)

# Merge with positioning and CSR data
if not positioning_df.empty:
    final_analysis = final_analysis.merge(
        positioning_df[['brand', 'market_segment']], 
        left_on='brand_clean', right_on='brand', how='left'
    ).drop('brand', axis=1)
    final_analysis = final_analysis.merge(
        csr_analysis[['brand', 'csr_score']], 
        left_on='brand_clean', right_on='brand', how='left'
    ).drop('brand', axis=1)
    
    print(f"   After positioning merge: {final_analysis.shape}")
    print(f"   Missing CSR scores: {final_analysis['csr_score'].isna().sum()}")
else:
    print("   ⚠️ No positioning data - using defaults")
    final_analysis['market_segment'] = 'Unknown'
    final_analysis['csr_score'] = 5.0

# Calculate strategic value score using RELATIONSHIP counts
# Normalize metrics to 0-10 scale
final_analysis['dealer_score'] = (final_analysis['dealer_count'] / final_analysis['dealer_count'].max()) * 10
final_analysis['rating_score'] = (final_analysis['avg_rating'] / 5) * 10  # assume 5 max rating
final_analysis['review_score'] = np.log1p(final_analysis['avg_reviews']) / np.log1p(final_analysis['avg_reviews'].max()) * 10

# Strategic value = business performance + CSR alignment
business_weights = {'dealer': 0.4, 'rating': 0.2, 'reviews': 0.2, 'csr': 0.2}
final_analysis['strategic_value'] = (
    business_weights['dealer'] * final_analysis['dealer_score'] +
    business_weights['rating'] * final_analysis['rating_score'] +
    business_weights['reviews'] * final_analysis['review_score'] +
    business_weights['csr'] * final_analysis['csr_score'].fillna(5)
)

final_analysis = final_analysis.sort_values('strategic_value', ascending=False)

print(f"\n🎯 CORRECTED Pon Brand Strategic Value Ranking (by relationships):")
strategic_display = final_analysis[['brand_clean', 'dealer_count', 'unique_locations', 'avg_rating', 
                                  'market_segment', 'csr_score', 'strategic_value']].round(1)
print(strategic_display.to_string(index=False))

# Generate recommendations based on strategic value and market position
recommendations = []
for _, brand in final_analysis.iterrows():
    brand_name = brand['brand_clean']
    relationships = brand['dealer_count']  # Now using relationships
    locations = brand['unique_locations']
    rating = brand['avg_rating']
    csr = brand['csr_score'] if pd.notna(brand['csr_score']) else 5.0
    strategic = brand['strategic_value']
    segment = brand['market_segment']
    
    # Decision logic based on RELATIONSHIPS
    if strategic >= 8.0 and relationships >= 50:
        action = "EXPAND"
        rationale = f"High strategic value ({strategic:.1f}) with strong network ({relationships} relationships). Core brand."
    elif strategic >= 6.0 and csr >= 7.0:
        action = "MAINTAIN"
        rationale = f"Good strategic value with high CSR impact. Sustainable growth focus."
    elif strategic >= 5.0 and relationships < 20:
        action = "EVALUATE"
        rationale = f"Limited scale ({relationships} relationships). Assess market potential vs investment."
    elif rating < 3.5 or strategic < 4.0:
        action = "CONSIDER EXIT"
        rationale = f"Low performance (rating {rating:.1f}, strategic {strategic:.1f}). Resource reallocation candidate."
    else:
        action = "MAINTAIN"
        rationale = f"Stable performance in {segment} segment. Monitor trends."
    
    recommendations.append({
        'brand': brand_name,
        'action': action,
        'rationale': rationale,
        'strategic_value': strategic,
        'csr_score': csr,
        'dealer_count': relationships,  # Relationships count
        'unique_locations': locations
    })

recommendations_df = pd.DataFrame(recommendations)

print(f"\n📋 CORRECTED Portfolio Strategy Recommendations:")
print("=" * 80)
for _, rec in recommendations_df.iterrows():
    print(f"\n{rec['brand'].upper()}: {rec['action']}")
    print(f"   {rec['rationale']}")
    print(f"   Strategic Value: {rec['strategic_value']:.1f}/10 | CSR Score: {rec['csr_score']:.1f}/10")
    print(f"   Relationships: {rec['dealer_count']} | Locations: {rec['unique_locations']}")

🔧 CORRECTED: Using relationship counts for strategic analysis
   Brand performance data shape: (8, 8)
   Brands in analysis: ['cannondale', 'cervelo', 'focus', 'gazelle', 'kalkhoff', 'santa_cruz', 'union', 'urban_arrow']
   After positioning merge: (8, 10)
   Missing CSR scores: 0

🎯 CORRECTED Pon Brand Strategic Value Ranking (by relationships):
brand_clean  dealer_count  unique_locations  avg_rating market_segment  csr_score  strategic_value
    gazelle           701               700         4.5     Mid-market        7.6              9.0
   kalkhoff           199               191         4.5     Mid-market        7.7              6.3
      union           247               247         4.4     Mid-market        6.7              6.2
urban_arrow           213               211         4.4        Premium        7.1              6.2
 cannondale            92                92         4.6     Mid-market        5.5              5.5
      focus             8                 8         4.9  

## Export Results & Visualizations

In [55]:
# Export analysis results
print("📁 Exporting Portfolio Analysis Results:")
print("=" * 50)

# 1. Brand performance overview
brand_performance.to_csv(OUTPUTS_DIR / 'tables/brand_performance_analysis.csv', index=False)
print(f"✅ Exported: brand_performance_analysis.csv")

# 2. Market positioning
if not positioning_df.empty:
    positioning_df.to_csv(OUTPUTS_DIR / 'tables/brand_market_positioning.csv', index=False)
    print(f"✅ Exported: brand_market_positioning.csv")

# 3. CSR scores
if not csr_analysis.empty:
    csr_analysis.to_csv(OUTPUTS_DIR / 'tables/brand_csr_analysis.csv', index=False)
    print(f"✅ Exported: brand_csr_analysis.csv")

# 4. Final recommendations
recommendations_df.to_csv(OUTPUTS_DIR / 'tables/portfolio_recommendations.csv', index=False)
print(f"✅ Exported: portfolio_recommendations.csv")

# 5. Cannibalization analysis
if len(multi_brand_locations) > 0:
    multi_brand_locations.to_csv(OUTPUTS_DIR / 'tables/multi_brand_dealers.csv', index=False)
    print(f"✅ Exported: multi_brand_dealers.csv")
if len(multi_pon_locations) > 0:
    multi_pon_locations.to_csv(OUTPUTS_DIR / 'tables/multi_pon_dealers.csv', index=False)
    print(f"✅ Exported: multi_pon_dealers.csv")

# FIXED: Create summary for board presentation with proper variable handling
pon_dealers = dealers[dealers['is_pon_dealer']]  # Get Pon dealers subset
executive_summary = {
    'total_pon_brands': len(brand_performance),
    'total_pon_dealers': int(brand_performance['unique_locations'].sum()),
    'total_pon_relationships': int(brand_performance['total_relationships'].sum()),
    'avg_pon_rating': float(brand_performance['avg_rating'].mean()),
    'true_market_share_pct': 22.3,  # From corrected calculation above
    'top_performing_brand': final_analysis.iloc[0]['brand_clean'],
    'highest_csr_brand': csr_analysis.iloc[0]['brand'] if not csr_analysis.empty else 'N/A',
    'expand_recommendations': list(recommendations_df[recommendations_df['action'] == 'EXPAND']['brand']),
    'exit_candidates': list(recommendations_df[recommendations_df['action'] == 'CONSIDER EXIT']['brand']),
    'cannibalization_risk': f"{len(multi_pon_locations)}/{len(pon_dealers)} Pon dealers ({pon_cannibalization_rate:.1f}%)",
    'market_segments_covered': list(positioning_df['market_segment'].unique()) if not positioning_df.empty else ['Unknown']
}

import json
with open(OUTPUTS_DIR / 'tables/portfolio_executive_summary.json', 'w') as f:
    json.dump(executive_summary, f, indent=2)
print(f"✅ Exported: portfolio_executive_summary.json")

print(f"\n🎯 Executive Summary:")
print(f"   Total Pon brands analyzed: {executive_summary['total_pon_brands']}")
print(f"   Total Pon dealer locations: {executive_summary['total_pon_dealers']:,}")
print(f"   Total Pon relationships: {executive_summary['total_pon_relationships']:,}")
print(f"   TRUE market share: {executive_summary['true_market_share_pct']:.1f}% (corrected)")
print(f"   Top strategic brand: {executive_summary['top_performing_brand']}")
print(f"   Highest CSR impact: {executive_summary['highest_csr_brand']}")
print(f"   Expansion candidates: {', '.join(executive_summary['expand_recommendations'])}")
if executive_summary['exit_candidates']:
    print(f"   Exit candidates: {', '.join(executive_summary['exit_candidates'])}")
print(f"   Multi-Pon dealer risk: {executive_summary['cannibalization_risk']}")

📁 Exporting Portfolio Analysis Results:
✅ Exported: brand_performance_analysis.csv
✅ Exported: brand_market_positioning.csv
✅ Exported: brand_csr_analysis.csv
✅ Exported: portfolio_recommendations.csv
✅ Exported: multi_brand_dealers.csv
✅ Exported: multi_pon_dealers.csv
✅ Exported: portfolio_executive_summary.json

🎯 Executive Summary:
   Total Pon brands analyzed: 8
   Total Pon dealer locations: 1,496
   Total Pon relationships: 1,507
   TRUE market share: 22.3% (corrected)
   Top strategic brand: gazelle
   Highest CSR impact: kalkhoff
   Expansion candidates: gazelle
   Multi-Pon dealer risk: 391/978 Pon dealers (40.0%)


## Create Portfolio Visualization

In [56]:
# Create comprehensive portfolio visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Strategic Value vs CSR Impact',
        'Market Positioning (Dealer Count vs Rating)',
        'Brand Performance Scores',
        'Portfolio Recommendations'
    ),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# 1. Strategic Value vs CSR Impact scatter
colors = {'EXPAND': 'green', 'MAINTAIN': 'blue', 'EVALUATE': 'orange', 'CONSIDER EXIT': 'red'}
for action in recommendations_df['action'].unique():
    subset = recommendations_df[recommendations_df['action'] == action]
    fig.add_trace(
        go.Scatter(
            x=subset['strategic_value'],
            y=subset['csr_score'],
            mode='markers+text',
            marker=dict(size=subset['dealer_count']/3, color=colors.get(action, 'gray')),
            text=subset['brand'],
            textposition='top center',
            name=action,
            hovertemplate='<b>%{text}</b><br>Strategic: %{x:.1f}<br>CSR: %{y:.1f}<br>Dealers: %{marker.size}<extra></extra>'
        ),
        row=1, col=1
    )

# 2. Market positioning (dealers vs rating)
fig.add_trace(
    go.Scatter(
        x=final_analysis['dealer_count'],
        y=final_analysis['avg_rating'],
        mode='markers+text',
        marker=dict(size=final_analysis['strategic_value'], 
                   color=final_analysis['strategic_value'],
                   colorscale='Viridis',
                   showscale=False),
        text=final_analysis['brand_clean'],
        textposition='top center',
        showlegend=False
    ),
    row=1, col=2
)

# 3. Performance scores bar chart
fig.add_trace(
    go.Bar(
        x=final_analysis['brand_clean'],
        y=final_analysis['strategic_value'],
        marker_color=final_analysis['strategic_value'],
        marker_colorscale='RdYlGn',
        showlegend=False,
        text=final_analysis['strategic_value'].round(1),
        textposition='outside'
    ),
    row=2, col=1
)

# 4. Recommendations summary
rec_counts = recommendations_df['action'].value_counts()
fig.add_trace(
    go.Bar(
        x=rec_counts.index,
        y=rec_counts.values,
        marker_color=[colors.get(action, 'gray') for action in rec_counts.index],
        showlegend=False,
        text=rec_counts.values,
        textposition='outside'
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="Pon Brand Portfolio Analysis - Strategic Recommendations",
    showlegend=True
)

# Update axes labels
fig.update_xaxes(title_text="Strategic Value", row=1, col=1)
fig.update_yaxes(title_text="CSR Score", row=1, col=1)

fig.update_xaxes(title_text="Dealer Count", row=1, col=2)
fig.update_yaxes(title_text="Avg Rating", row=1, col=2)

fig.update_xaxes(title_text="Brand", tickangle=45, row=2, col=1)
fig.update_yaxes(title_text="Strategic Value", row=2, col=1)

fig.update_xaxes(title_text="Recommendation", row=2, col=2)
fig.update_yaxes(title_text="Number of Brands", row=2, col=2)

# Save visualization
fig.write_html(OUTPUTS_DIR / 'plots/portfolio_analysis.html')
try:
    fig.write_image(OUTPUTS_DIR / 'plots/portfolio_analysis.png', width=1200, height=800)
    print("📊 Portfolio visualization saved as HTML and PNG")
except:
    print("📊 Portfolio visualization saved as HTML (PNG export failed)")

print(f"\n✅ Portfolio Analysis Complete!")
print(f"   All results exported to: {OUTPUTS_DIR / 'tables'}")
print(f"   Visualizations saved to: {OUTPUTS_DIR / 'plots'}")
print(f"\n🎯 Ready for board presentation with data-driven portfolio strategy!")

📊 Portfolio visualization saved as HTML (PNG export failed)

✅ Portfolio Analysis Complete!
   All results exported to: ../outputs/tables
   Visualizations saved to: ../outputs/plots

🎯 Ready for board presentation with data-driven portfolio strategy!


In [57]:
# Test the fixed data loading in notebook 06
import pandas as pd
import numpy as np
from pathlib import Path

# Setup paths
DATA_DIR = Path('../data')

# Load data
dealers = pd.read_parquet(DATA_DIR / 'processed/dealers.parquet')
dealers_all_brands = pd.read_parquet(DATA_DIR / 'processed/dealers_all_brands.parquet')

print("Testing data loading fix:")
print(f"dealers shape: {dealers.shape}")
print(f"dealers_all_brands shape: {dealers_all_brands.shape}")

print(f"\nColumns in dealers: {list(dealers.columns)}")
print(f"Columns in dealers_all_brands: {list(dealers_all_brands.columns)}")

# Test the FIX: Merge PC4 column BEFORE grouping
dealers_with_pc4 = dealers_all_brands.merge(
    dealers[['google_place_id', 'pc4']], 
    on='google_place_id', 
    how='left'
)

print(f"\nAfter merge:")
print(f"dealers_with_pc4 shape: {dealers_with_pc4.shape}")
print(f"PC4 column added: {'pc4' in dealers_with_pc4.columns}")

# Test groupby now works
dealer_pc4_relationships = dealers_with_pc4.groupby(['pc4', 'brand_clean']).size().reset_index(name='relationships_per_pc4')
print(f"\nGroupby successful: {dealer_pc4_relationships.shape}")
print("✅ Fix confirmed - PC4 column available for grouping after merge")

Testing data loading fix:
dealers shape: (2080, 21)
dealers_all_brands shape: (6748, 17)

Columns in dealers: ['google_place_id', 'name', 'google_name', 'brand_clean', 'is_pon_dealer', 'google_rating', 'google_user_ratings_total', 'google_lat', 'google_lng', 'postal_code', 'google_address', 'country', 'google_link', 'website', 'house_number', 'street', 'pon_brand_count', 'total_brand_count', 'brands_sold', 'brand', 'pc4']
Columns in dealers_all_brands: ['name', 'brand', 'website', 'google_place_id', 'house_number', 'street', 'country', 'postal_code', 'google_name', 'google_address', 'google_rating', 'google_user_ratings_total', 'google_lat', 'google_lng', 'google_link', 'brand_clean', 'is_pon_dealer']

After merge:
dealers_with_pc4 shape: (6748, 18)
PC4 column added: True

Groupby successful: (6014, 3)
✅ Fix confirmed - PC4 column available for grouping after merge
